In [ ]:
import pandas as pd

#df = pd.read_csv("all_results_sort_26_02_2.csv")
#df = pd.read_csv("all_results_median_rank_28_02.csv")
#df = pd.read_csv("all_results_sum_28_02.csv")
#df = pd.read_csv("all_results_gaussian_sort_19_03.csv")
# df = pd.read_csv("results_summarised/all_results_gaussian_sort_02_04.csv")
# old_df = pd.read_csv("results_summarised/all_results_gaussian_sort_19_03.csv")
# old_df["old"]=1
# df["old"]=0
# df = pd.concat([df, old_df], ignore_index=True)
# df = pd.concat([df, pd.read_csv("results_summarised/all_results_sort_ff_02_04.csv")])
# df = pd.concat([df, pd.read_csv("results_summarised/all_results_sort_26_02_2.csv")])
# df = pd.concat([pd.read_csv("results_summarised/all_results_sort_with_holes_03_04.csv"),
#                 pd.read_csv("results_summarised/all_results_sort_04_04.csv")])
#df = pd.read_csv("results_summarised/all_results_normalization_08_04.csv")

#df = pd.concat([pd.read_csv("results_summarised/all_results_sort_08_04_revert.csv"),
#                pd.read_csv("results_summarised/all_results_sort_06_04_full.csv")])
#df  = pd.read_csv("results_summarised/all_results_sort_06_04_full.csv")
#df  = pd.read_csv("results_summarised/all_new_results_sort_14_05_full.csv")
df  = pd.read_csv("results_summarised/sort_different_depth_17_05_2.csv")



In [ ]:
import ast
import pandas as pd

# Assuming df is your DataFrame and 'test_loss_list' is the column you're working with.

for index, row in df.iterrows():
    try:
        # Try to evaluate the row as a literal.
        pd.eval(row['test_loss_list'])
    except Exception as e:
        # If an exception occurs, print the index and the value that failed
        print(f"Failed at index {index}: {row['test_loss_list']}")
        print(f"Error: {e}")
        # Optionally, break after the first failure
        #break

In [ ]:
import ast
# replace "nan" with None
df['test_loss_list'] = df['test_loss_list'].str.replace('nan', 'None')
df['loss_list'] = df['loss_list'].str.replace('nan', 'None')
#df['test_loss_list'] = df['test_loss_list'].apply(lambda s: pd.eval(s))
#df['loss_list'] = df['loss_list'].apply(lambda s: pd.eval(s))

df['test_loss_list'] = df['test_loss_list'].apply(ast.literal_eval)
df['loss_list'] = df['loss_list'].apply(ast.literal_eval)

# Create a new DataFrame for Plotly Express
rows_list = []
for _, row in df.iterrows():
    params = row.to_dict()
    test_loss_list = params.pop('test_loss_list')
    loss_list = params.pop('loss_list')
    for epoch, loss in enumerate(test_loss_list):
        params['epoch'] = epoch
        params['test_loss'] = loss
        params['train_loss'] = loss_list[epoch]
        rows_list.append(params.copy())  # Use .copy() to avoid modifying the original params

# Create a new DataFrame
df_long = pd.DataFrame(rows_list)
df_long = df_long.fillna("missing")

In [ ]:
# make embedding_function "gaussian" or "none"

def convert_embedding(name):
    if "GaussianEmbedder" in name:
        return "gaussian"
    elif "PositionalEncoding" in name:
        return "positional"
    elif "RandomFeature" in name:
        return "fourier"
    else:
        return "none"
df_long["embedding"] = df_long["embedding_function"].apply(convert_embedding)

# remove Unnamed: 0
df_long = df_long.drop(columns=["Unnamed: 0", "job_id"])

In [ ]:
# average over iter
# fill na with "missing"
df_long = df_long.fillna("missing")
# convert all to string except test_loss
# removing rows where test_loss is missing
df_long = df_long[df_long["test_loss"] != "missing"]
# same for epoch
df_long = df_long[df_long["epoch"] != "missing"]
df_long = df_long.applymap(str)
df_long["test_loss"] = df_long["test_loss"].astype(float)
df_long["epoch"] = df_long["epoch"].astype(int)
df_long["train_loss"] = df_long["train_loss"].astype(float)
cols = list(df_long.columns)
# # remove iter and test_loss
cols.remove("iter")
cols.remove("test_loss")
cols.remove("train_loss")
# check that we have 5 rows for each combination of the other columns
#assert all(df_long.groupby(cols).size() > 5)
#show groups with less than 5
#df_long.groupby(cols).size().sort_values()
#df_long = df_long.groupby(cols)["test_loss"].mean().reset_index()
# #add a column for the number of rows and the mean test_loss
# df_long = df_long.groupby(cols).agg(
#     test_loss=('test_loss', 'median'),  # Calculate mean test_loss
#     train_loss=('train_loss', 'median'),  # Calculate mean test_loss
#     count=('test_loss', 'count'),           # Count the number of entries used for the mean
#     std=('test_loss', 'std')
# ).reset_index()


In [ ]:
import numpy as np
filter_dic = {
    "n_layer": 8,
    "n_head": 4,
    "lr": 1e-4,
    #"embedding": "none",
    #"lr": 1e-4,
    #"task": "sort_with_holes",
    "task": "sort",
    "seq_length": 10,
    #"n_embd": 128,
    #"read_in_bias": False,
    "attention_only": True,
    #"revert_embedding": True,
    #"parametrize_std": False,
    #"n_epochs": 130,
    #"n_epochs": 300,
}



df_plot = df_long.copy()
for key, value in filter_dic.items():
    df_plot = df_plot[np.isin(df_plot[key], [value, "missing"])]

print(len(df_plot))


In [ ]:
df_long["revert_embedding"].unique()

In [ ]:
#df_plot = df_plot[(df_plot["iter"] == "4") & (df_plot["embedding"] == "gaussian")]
df_plot = df_plot[df_plot["embedding"] != "positional"]
# remove embedding_function
#df_plot = df_plot.drop(columns=["embedding_function"])

# replace length_scale by the bandwidth column when length_scale is missing
# if length_scale column do not exist, first create it
if 'length_scale' not in df_plot.columns:
    df_plot['length_scale'] = 'missing'
#df_plot['length_scale'] = df_plot['length_scale'].replace('missing', pd.NA).fillna(df['bandwidth'])
#df_plot.reset_index(drop=True, inplace=True)
mask = df_plot['length_scale'] == 'missing'


# Replace 'missing' values in 'length_scale' with the corresponding value from 'bandwidth'
df_plot.loc[mask, 'length_scale'] = df_plot.loc[mask, 'bandwidth']

df_plot["embedding_kernel_length"] = df_plot["embedding"] + " " + df_plot["kernel"] + " " + df_plot["length_scale"]

#df_plot = df_plot[df_plot["epoch"] < 101]
#df_plot = df_plot[df_plot["count"] >= 5]
#df_plot = df_plot[np.isin(df_plot["seq_length"], ["3", "5", "7"])]

In [ ]:
import plotly.express as px
df_box = df_plot.copy()
#df_box = df_box[df_box["epoch"] == 299]
#df_box = df_box[df_box["embedding"] == "none"]
#df_box = df_box[np.isin(df_box["length_scale"], ["[1.0]", "[2.0]"])]
fig = px.scatter(
    df_box,
    y = "test_loss",
    x ='epoch',
    log_y=True,
    color='embedding_kernel_length',
    hover_data=df_plot.columns,
    #facet_row="length_scale",
    #facet_row="holes",
    #facet_row="parametrize_std",
    height=1500,
    # set order on holes
    #category_orders={"holes": ["{}", "[(0, 1), (2, 3), (4, 5), (6, 7), (8, 9)]", "[(3, 7)]"]},
    # show std
    #error_y="std",
    # transparent

    # don't share y-axes

    
    #facet_row="seq_length",
    # limit x to 100
    #facet_row="attention_only",
    #title=f"n_layer={filter_dic['n_layer']}, n_head={filter_dic['n_head']}, lr={filter_dic['lr']}, n_embd={filter_dic['n_embd']}",
)


fig.show()
# log scale on y-axis


In [ ]:
import plotly.express as px
df_box = df_plot.copy()
#df_box = df_box[df_box["epoch"] == 299]
#df_box = df_box[df_box["holes"] == "missing"]
#df_box = df_box[np.isin(df_box["length_scale"], ["[1.0]", "[2.0]"])]
df_box["rmse_test"] = np.sqrt(df_box["test_loss"])
fig = px.scatter(
    df_box,
    y = "rmse_test",
    x ='epoch',
    log_y=True,
    color='embedding_kernel_length',
    hover_data=df_plot.columns,
    #facet_row="length_scale",
    #facet_row="holes",
    #facet_row="parametrize_std",
    height=1500,
    # set order on holes
    #category_orders={"holes": ["{}", "[(0, 1), (2, 3), (4, 5), (6, 7), (8, 9)]", "[(3, 7)]"]},
    # show std
    #error_y="std",
    # transparent

    # don't share y-axes

    
    #facet_row="seq_length",
    # limit x to 100
    #facet_row="attention_only",
    #title=f"n_layer={filter_dic['n_layer']}, n_head={filter_dic['n_head']}, lr={filter_dic['lr']}, n_embd={filter_dic['n_embd']}",
)


fig.show()
# log scale on y-axis


In [ ]:
df_box["parametrize_std"].unique()

In [ ]:
import plotly.express as px
df_box = df_plot.copy()
df_box = df_box[df_box["epoch"] == 100]
#df_box = df_box[np.isin(df_box["length_scale"], ["[1.0]", "[2.0]"])]
fig = px.box(
    df_box,
    y = "embedding_kernel_length",
    x ='test_loss',
    log_x=True,
    color='embedding',
    hover_data=df_plot.columns,
    #facet_row="length_scale",
    facet_row="n_embd",
    facet_col="parametrize_std",
    #facet_row="parametrize_std",
    height=1500,
    # set order on holes
    category_orders={"holes": ["{}", "[(0, 1), (2, 3), (4, 5), (6, 7), (8, 9)]", "[(3, 7)]"]},
    # show std
    #error_y="std",
    # transparent

    # don't share y-axes

    
    #facet_row="seq_length",
    # limit x to 100
    #facet_row="attention_only",
    #title=f"n_layer={filter_dic['n_layer']}, n_head={filter_dic['n_head']}, lr={filter_dic['lr']}, n_embd={filter_dic['n_embd']}",
)


fig.show()
# log scale on y-axis


In [ ]:
(15 / 128) * 5

In [ ]:
df_box

In [ ]:
import plotly.express as px
df_box = df_plot.copy()
df_box = df_box[df_box["epoch"] == 127]
#df_box = df_box[np.isin(df_box["length_scale"], ["[1.0]", "[2.0]"])]
df_box["rmse_test"] = np.sqrt(df_box["test_loss"])
fig = px.box(
    df_box,
    y = "embedding_kernel_length",
    x ='rmse_test',
    log_x=True,
    color='embedding',
    hover_data=df_plot.columns,
    #facet_row="length_scale",
    facet_row="holes",
    #facet_row="parametrize_std",
    height=1500,
    # set order on holes
    category_orders={"holes": ["{}", "[(0, 1), (2, 3), (4, 5), (6, 7), (8, 9)]", "[(3, 7)]"]},
    # show std
    #error_y="std",
    # transparent

    # don't share y-axes

    
    #facet_row="seq_length",
    # limit x to 100
    #facet_row="attention_only",
    #title=f"n_layer={filter_dic['n_layer']}, n_head={filter_dic['n_head']}, lr={filter_dic['lr']}, n_embd={filter_dic['n_embd']}",
)


fig.show()
# log scale on y-axis


In [ ]:
df_box["length_scale"]

In [ ]:
import plotly.express as px
import numpy as np


results_to_plot = df_plot.copy()
results_to_plot["epoch"] = results_to_plot["epoch"].astype(float)
#results_to_plot["test_loss"] = np.sqrt(results_to_plot["test_loss"]) # for rmse #TODO
filter_dic = {
    #"holes": "{}",
    #"fourier_length_scale": 0.1,
    #"fourier_length_scale": "0.0520833333333333",
    #"gaussian_embedding_sd": "[0.052083333333333336]",
    #"gaussian_embedding_sd": 0.1,
    #"epoch": "30",
}

# rename "none" embedding to "linear"
results_to_plot["embedding"] = results_to_plot["embedding"].replace("none", "linear")

# allow missing or value in filter_dic
for key, value in filter_dic.items():
    results_to_plot = results_to_plot[np.isin(results_to_plot[key], [value, "missing"])]

# if embedding is fourier, only keep "fourier_length_scale": "0.0520833333333333",
results_to_plot = results_to_plot[(results_to_plot["embedding"] != "fourier") | (results_to_plot["length_scale"] == "0.46875")]
# same for gaussian with "[0.052083333333333336]"
results_to_plot = results_to_plot[(results_to_plot["embedding"] != "gaussian") | (results_to_plot["length_scale"] == "[0.5859375]")]

# fix the palette for the colors
palette = px.colors.qualitative.Plotly
palette = np.array(palette)

# remove gaussian embedding
#results_to_plot = results_to_plot[results_to_plot["embedding_type"] != "gaussian"]


#group_cols = ["epoch", "dim_feedforward", "dropout", "norm_first", "embedding_type"]
group_cols = results_to_plot.columns
#group_cols.remove(["loss_mlm", "loss_num", "loss_total", "iter"])
group_cols = [col for col in group_cols if col not in ["train_loss", "test_loss", "epoch"]]
import pandas as pd


# Define the threshold
threshold = 0.1

# Group by the specified columns and filter
def min_epoch_below_threshold(group):
    below_threshold = group[group['test_loss'] < threshold]
    if not below_threshold.empty:
        return below_threshold['epoch'].min()
    return None  # Return None if no epochs are below the threshold

# Group by the specified columns and apply the function
#result = results_to_plot.groupby(group_cols).apply(min_epoch_below_threshold).reset_index()
result = results_to_plot.groupby(group_cols).apply(lambda group: group[group['test_loss'] < threshold]['epoch'].min()).reset_index()

# Rename the result column for clarity
result.rename(columns={0: 'min_epoch_to_reach_threshold'}, inplace=True)
result["threshold"] = threshold

# Display the result
print(result)

threshold = 0.01
# Group by the specified columns and filter
result2 = results_to_plot.groupby(group_cols).apply(lambda group: group[group['test_loss'] < threshold]['epoch'].min()).reset_index()

# Rename the result column for clarity
result2.rename(columns={0: 'min_epoch_to_reach_threshold'}, inplace=True)
result2["threshold"] = threshold

threshold = 0.005
# Group by the specified columns and filter
result3 = results_to_plot.groupby(group_cols).apply(lambda group: group[group['test_loss'] < threshold]['epoch'].min()).reset_index()

# Rename the result column for clarity
result3.rename(columns={0: 'min_epoch_to_reach_threshold'}, inplace=True)
result3["threshold"] = threshold

# Display the result
#fig = px.scatter(result, x='min_epoch_to_reach_threshold', y="embedding_type", title='Minimum epoch to reach threshold', height=900, color='min_epoch_to_reach_threshold', facet_col='dataset_path',
#                 hover_data=result.columns)
#fig.show()

In [ ]:
results_to_plot.embedding.unique()

In [ ]:
# combine the two results
import pandas as pd


# Ensure 'group_cols' are set correctly as per your earlier code
group_cols = results_to_plot.columns
group_cols = [col for col in group_cols if col not in ["train_loss", "test_loss", "epoch", "min_epoch_to_reach_threshold"]]

# Merge results and results2 on group columns
merged_results = pd.concat([result, result2, result3])
# Calculate standard deviation, min, and max for the epoch columns
import pandas as pd

# Assuming merged_results is already defined and contains the necessary columns

# Calculate mean, std, min, and max grouped by 'embedding_type'
grouped_stats = merged_results.groupby(['embedding', "threshold"]).agg({
    'min_epoch_to_reach_threshold': ['mean', 'std', 'min', 'max'],
}).reset_index().reset_index()

# Flattening the column MultiIndex
grouped_stats.columns = [''.join(col).strip() for col in grouped_stats.columns.values]



# # Function to format the entries with mean (min-max)
def format_entry(row):
     return f"{row['min_epoch_to_reach_thresholdmean']:.1f} ({row['min_epoch_to_reach_thresholdmin']}-{row['min_epoch_to_reach_thresholdmax']})"

    
grouped_stats["formatted_min_epochs"] = grouped_stats.apply(format_entry, axis=1)

# pivot formatted_min_epochs by threshold
grouped_stats = grouped_stats.pivot(index='embedding', columns='threshold', values='formatted_min_epochs').reset_index()

# reorder and rename columns
grouped_stats = grouped_stats[["embedding", 0.1, 0.01, 0.005]]
grouped_stats.columns = ["Embedding", "MSE=0.1", "MSE=0.01", "MSE=0.005"]

# export to latex
print(grouped_stats.to_latex(index=False, escape=False))







In [ ]:
grouped_stats.columns.values

In [ ]:
results_to_plot

In [ ]:
grouped_stats

In [ ]:
df_box[["n_layer", "n_head", "n_embd"]]

In [ ]:
import plotly.express as px
fig = px.line(
    df_plot,
    x='epoch',
    y='test_loss',
    log_y=True,
    color='normalization',
    hover_data=df_plot.columns,
    facet_row="length_scale",
    facet_col="holes",
    #facet_row="parametrize_std",
    height=1500,
    # show std
    #error_y="std",
    # transparent

    # don't share y-axes

    
    #facet_row="seq_length",
    # limit x to 100
    #facet_row="attention_only",
    #title=f"n_layer={filter_dic['n_layer']}, n_head={filter_dic['n_head']}, lr={filter_dic['lr']}, n_embd={filter_dic['n_embd']}",
)
fig.show()
# log scale on y-axis
